## Step 3: Save the Dictionary to SQLite3

In [ ]:
import sqlite3

In [ ]:
conn = sqlite3.connect('UMGC.db')

In [ ]:
c = conn.cursor()
c.execute('''
    CREATE TABLE classes (
        name TEXT,
        title TEXT,
        credit TEXT,
        description TEXT,
        prerequisites TEXT,
        recommended TEXT,
        warnings TEXT,
        substitutions TEXT,
        pre TEXT,
        pre_credits TEXT,
        pre_notes TEXT,
        done INTEGER
    )
''')


In [ ]:
# Insert data into the table

for class_name, class_info in classes.items():
    c.execute('''
        INSERT INTO classes VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        class_info['name'],
        class_info['title'],
        class_info['credit'],
        class_info['description'],
        class_info['prerequisites'],
        class_info['recommended'],
        class_info['warnings'],
        class_info['substitutions'],
        class_info['pre'],
        class_info['pre_credits'],
        class_info['pre_notes'],
        int(class_info['done'])
    ))


In [ ]:
conn.commit()
conn.close()

In [ ]:
json_data = df.to_json(orient='records')


In [ ]:
data_dict = df.to_dict(orient='records')


In [1]:
data_json_new = [
    { "seq":  1, "name": "PACE 111", "credits": 3, "type": "general",  "period":  1, "session": 1, "prerequisite": ""                                },
    { "seq":  2, "name": "LIBS 150", "credits": 1, "type": "general",  "period":  1, "session": 2, "prerequisite": ""                                },
    { "seq":  3, "name": "WRTG 111", "credits": 3, "type": "general",  "period":  1, "session": 3, "prerequisite": ""                                },
    { "seq":  4, "name": "WRTG 112", "credits": 3, "type": "general",  "period":  2, "session": 1, "prerequisite": ""                                },
    { "seq":  5, "name": "NUTR 100", "credits": 3, "type": "general",  "period":  2, "session": 2, "prerequisite": ""                                },
    { "seq":  6, "name": "BMGT 110", "credits": 3, "type": "major",    "period":  2, "session": 3, "prerequisite": ""                                },
    { "seq":  7, "name": "SPCH 100", "credits": 3, "type": "general",  "period":  3, "session": 1, "prerequisite": ""                                },
    { "seq":  8, "name": "STAT 200", "credits": 3, "type": "required", "period":  3, "session": 2, "prerequisite": ""                                },
    { "seq":  9, "name": "IFSM 300", "credits": 3, "type": "required", "period":  4, "session": 1, "prerequisite": ""                                },
    { "seq": 10, "name": "ACCT 220", "credits": 3, "type": "major",    "period":  4, "session": 1, "prerequisite": ""                                }, 
    { "seq": 11, "name": "HUMN 100", "credits": 3, "type": "general",  "period":  4, "session": 2, "prerequisite": ""                                }, 
    { "seq": 12, "name": "BIOL 103", "credits": 4, "type": "general",  "period":  5, "session": 1, "prerequisite": ""                                }, 
    { "seq": 13, "name": "ECON 201", "credits": 3, "type": "required", "period":  4, "session": 3, "prerequisite": ""                                }, 
    { "seq": 14, "name": "ARTH 334", "credits": 3, "type": "general",  "period":  5, "session": 2, "prerequisite": ""                                },
    { "seq": 15, "name": "ELECTIVE", "credits": 3, "type": "elective", "period":  6, "session": 1, "prerequisite": ""                                }, 
    { "seq": 16, "name": "ECON 203", "credits": 3, "type": "required", "period":  6, "session": 2, "prerequisite": ""                                }, 
    { "seq": 17, "name": "ACCT 221", "credits": 3, "type": "major",    "period":  6, "session": 3, "prerequisite": "ACCT 220"                        }, 
    { "seq": 18, "name": "ELECTIVE", "credits": 3, "type": "elective", "period":  7, "session": 1, "prerequisite": ""                                }, 
    { "seq": 19, "name": "BMGT 364", "credits": 3, "type": "major",    "period":  7, "session": 2, "prerequisite": ""                                }, 
    { "seq": 20, "name": "ELECTIVE", "credits": 3, "type": "elective", "period":  8, "session": 1, "prerequisite": ""                                }, 
    { "seq": 21, "name": "BMGT 365", "credits": 3, "type": "major",    "period":  8, "session": 2, "prerequisite": "BMGT 364"                        }, 
    { "seq": 22, "name": "ELECTIVE", "credits": 3, "type": "elective", "period":  8, "session": 3, "prerequisite": ""                                }, 
    { "seq": 23, "name": "MRKT 310", "credits": 3, "type": "major",    "period":  8, "session": 1, "prerequisite": ""                                }, 
    { "seq": 24, "name": "WRTG 394", "credits": 3, "type": "general",  "period":  9, "session": 1, "prerequisite": "WRTG 112"                        }, 
    { "seq": 25, "name": "ELECTIVE", "credits": 3, "type": "elective", "period":  9, "session": 2, "prerequisite": ""                                }, 
    { "seq": 26, "name": "BMGT 380", "credits": 3, "type": "major",    "period":  9, "session": 3, "prerequisite": ""                                }, 
    { "seq": 27, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 10, "session": 1, "prerequisite": ""                                }, 
    { "seq": 28, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 10, "session": 2, "prerequisite": ""                                }, 
    { "seq": 29, "name": "HRMN 300", "credits": 3, "type": "major",    "period": 10, "session": 3, "prerequisite": ""                                },
    { "seq": 30, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 11, "session": 1, "prerequisite": ""                                }, 
    { "seq": 31, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 11, "session": 2, "prerequisite": ""                                }, 
    { "seq": 32, "name": "FINC 330", "credits": 3, "type": "major",    "period": 12, "session": 1, "prerequisite": "ACCT 221 & STAT 200"             }, 
    { "seq": 33, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 12, "session": 2, "prerequisite": ""                                }, 
    { "seq": 34, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 12, "session": 3, "prerequisite": ""                                }, 
    { "seq": 35, "name": "BMGT 496", "credits": 3, "type": "major",    "period": 12, "session": 1, "prerequisite": ""                                }, 
    { "seq": 36, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 13, "session": 1, "prerequisite": ""                                }, 
    { "seq": 37, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 13, "session": 2, "prerequisite": ""                                }, 
    { "seq": 38, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 13, "session": 3, "prerequisite": ""                                }, 
    { "seq": 39, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 14, "session": 1, "prerequisite": ""                                }, 
    { "seq": 40, "name": "BMGT 495", "credits": 3, "type": "major",    "period": 14, "session": 2, "prerequisite": "BMGT 365 & MRKT 310 & FINC 330"  }, 
    { "seq": 41, "name": "CAPSTONE", "credits": 1, "type": "elective", "period": 14, "session": 3, "prerequisite": "FINC 330"                        }, 
]


## Updated Python Code

In [107]:
import pandas as pd
import numpy as np

def generate_header_data(start_semester, num_periods, data_df = df):
    seasons = ['WINTER', 'SPRING', 'SUMMER', 'FALL']
    semester_data = []
    start_season, start_year = start_semester.split(' ')
    start_year = int(start_year)
    season_index = seasons.index(start_season)
    year = start_year
    period = 0

    while period < num_periods:
        for j in range(season_index, len(seasons)):
            semester_data.append(f'{seasons[j]} {year}')
            period += 1

            # Break the loop when i equals num_periods
            if period == num_periods:
                break

        # Reset the season index to start from 'WINTER' for the next year
        season_index = 0
        year += 1

    df = pd.DataFrame(semester_data, columns=['term'])
    df['width'] = df['term'].apply(lambda x: 190 if 'SUMMER' in x else 260)
    df['offset'] = df['term'].apply(lambda x: 2 if 'SUMMER' in x else 3)
    df['fontsize'] = '14px'
    df['description'] = ''
    df['space'] = 40
    df['xpos'] = df['width'] + df['space']

    x0 = 10
    # Calculate the cumulative sum of 'xpos'
    df['x'] = df['xpos'].cumsum()
    df['x'] = df['x'].shift(1)
    df.loc[0, 'x'] = 0
    df['x'] = df['x'] + x0
    df['y'] = 10
    df['color'] = 'lightgray'
    df['textcolor'] = 'black'
    df['period'] = np.arange(1, num_periods+1)

    df.drop
    # Sum credits per period and convert to a DataFrame
    total_credits = data_df.groupby('period')['credits'].sum().sort_index()
    total_credits_df = total_credits.reset_index()

    df = pd.merge(df, total_credits_df, on='period', how='inner')
    df['name'] = df['term']
    df['printname'] = df['name'] + ' (' + df['credits'].astype(str) + ')'

    return df[['x', 'y', 'width', 'printname', 'color', 'textcolor', 'offset', 'fontsize', 'period', 'name', 'credits', 'description']]

def set_colors(row):
    if row['type'] == 'general':
        return pd.Series(['green', 'white'])
    elif row['type'] == 'major':
        return pd.Series(['blue', 'white'])
    elif row['type'] == 'required':
        return pd.Series(['red', 'white'])
    elif row['type'] == 'elective':
        return pd.Series(['yellow', 'black'])
    else:
        return pd.Series(['white', 'black'])  # default colors

def prepare_d3_data(df, start_term='SPRING 2024'):
    # Function to prepare the data to be fed into the D3 code
    max_period = max(df['period'])
    headers = generate_header_data(start_term, max_period)

    df['description'] = df['prerequisite']
    df['width'] = 120
    # Calculate 'x' column
    df = pd.merge(df, headers[['period','x']], on='period', how='left')
    df['x'] += 70*(df['session']-1)

    # Calculate 'y' column
    df = df.sort_values(by=['period', 'session', 'seq' ])
    df['y_row'] = df.groupby('period').cumcount() + 1
    df['y'] = 70 + 45 * (df['y_row'] - 1)

    # Create rectangle colors
    df[['color', 'textcolor']] = df.apply(set_colors, axis=1)

    # Set text offset multiplier to 1 and text fontsize
    df['offset'] = 1
    df['fontsize'] = '12px'
    df['printname'] = df['name'] + ' (' + df['credits'].astype(str) + ')'
    
    df = df[['x', 'y', 'width', 'printname', 'color', 'textcolor', 'offset', 'fontsize', 'period', 'session', 'type', 'name', 'credits', 'description']]

    return df, headers

#### Get data from the database instead of the json file

In [112]:
def prepare_d3_data(df, start_term='SPRING 2024'):

    def set_colors(row):
        if row['type'] == 'general':
            return pd.Series(['green', 'white'])
        elif row['type'] == 'major':
            return pd.Series(['blue', 'white'])
        elif row['type'] == 'required':
            return pd.Series(['red', 'white'])
        elif row['type'] == 'elective':
            return pd.Series(['yellow', 'black'])
        else:
            return pd.Series(['white', 'black'])  # default colors

    def generate_header_data(start_semester, num_periods, data_df = df):
        seasons = ['WINTER', 'SPRING', 'SUMMER', 'FALL']
        semester_data = []
        start_season, start_year = start_semester.split(' ')
        start_year = int(start_year)
        season_index = seasons.index(start_season)
        year = start_year
        period = 0

        while period < num_periods:
            for j in range(season_index, len(seasons)):
                semester_data.append(f'{seasons[j]} {year}')
                period += 1

                # Break the loop when i equals num_periods
                if period == num_periods:
                    break

            # Reset the season index to start from 'WINTER' for the next year
            season_index = 0
            year += 1

        df = pd.DataFrame(semester_data, columns=['term'])
        df['width'] = df['term'].apply(lambda x: 190 if 'SUMMER' in x else 260)
        df['offset'] = df['term'].apply(lambda x: 2 if 'SUMMER' in x else 3)
        df['fontsize'] = '14px'
        df['description'] = ''
        df['space'] = 40
        df['xpos'] = df['width'] + df['space']

        x0 = 10
        # Calculate the cumulative sum of 'xpos'
        df['x'] = df['xpos'].cumsum()
        df['x'] = df['x'].shift(1)
        df.loc[0, 'x'] = 0
        df['x'] = df['x'] + x0
        df['y'] = 10
        df['color'] = 'lightgray'
        df['textcolor'] = 'black'
        df['period'] = np.arange(1, num_periods+1)

        df.drop
        # Sum credits per period and convert to a DataFrame
        total_credits = data_df.groupby('period')['credits'].sum().sort_index()
        total_credits_df = total_credits.reset_index()

        df = pd.merge(df, total_credits_df, on='period', how='inner')
        df['name'] = df['term']
        df['printname'] = df['name'] + ' (' + df['credits'].astype(str) + ')'

        return df[['x', 'y', 'width', 'printname', 'color', 'textcolor', 'offset', 
                   'fontsize', 'period', 'name', 'credits', 'description']]

    # Prepare data for the D3 figure
    
    max_period = max(df['period'])
    headers = generate_header_data(start_term, max_period)

    df['description'] = df['prerequisite']
    df['width'] = 120
    # Calculate 'x' column
    df = pd.merge(df, headers[['period','x']], on='period', how='left')
    df['x'] += 70*(df['session']-1)

    # Calculate 'y' column
    df = df.sort_values(by=['period', 'session', 'seq' ])
    df['y_row'] = df.groupby('period').cumcount() + 1
    df['y'] = 70 + 45 * (df['y_row'] - 1)

    # Create rectangle colors
    df[['color', 'textcolor']] = df.apply(set_colors, axis=1)

    # Set text offset multiplier to 1 and text fontsize
    df['offset'] = 1
    df['fontsize'] = '12px'
    df['printname'] = df['name'] + ' (' + df['credits'].astype(str) + ')'
    
    df = df[['x', 'y', 'width', 'printname', 'color', 'textcolor', 'offset', 'fontsize', 'period', 'session', 'type', 'name', 'credits', 'description']]

    return df, headers


In [113]:
df = pd.DataFrame(data_json_new)

# for new student, set course completion to False
# will pick up completed classes in database and from transfer credits
# all code will have to ensure we are working on pending courses and filter those where complete='False'
#
# Will need to do something similar for lock=True when the semester is assigned and we do not want to move it
# 

#df['complete'] = False

In [114]:
start_term = 'SPRING 2024'
df, headers = prepare_d3_data(df, start_term)

In [115]:
headers

,x,y,width,printname,color,textcolor,offset,fontsize,period,name,credits,description
0,10.0,10,260,SPRING 2024 (7),lightgray,black,3,14px,1,SPRING 2024,7,
1,310.0,10,190,SUMMER 2024 (9),lightgray,black,2,14px,2,SUMMER 2024,9,
2,540.0,10,260,FALL 2024 (6),lightgray,black,3,14px,3,FALL 2024,6,
3,840.0,10,260,WINTER 2025 (12),lightgray,black,3,14px,4,WINTER 2025,12,
4,1140.0,10,260,SPRING 2025 (7),lightgray,black,3,14px,5,SPRING 2025,7,
5,1440.0,10,190,SUMMER 2025 (9),lightgray,black,2,14px,6,SUMMER 2025,9,
6,1670.0,10,260,FALL 2025 (6),lightgray,black,3,14px,7,FALL 2025,6,
7,1970.0,10,260,WINTER 2026 (12),lightgray,black,3,14px,8,WINTER 2026,12,
8,2270.0,10,260,SPRING 2026 (9),lightgray,black,3,14px,9,SPRING 2026,9,
9,2570.0,10,190,SUMMER 2026 (9),lightgray,black,2,14px,10,SUMMER 2026,9,


In [116]:
df

,x,y,width,printname,color,textcolor,offset,fontsize,period,session,type,name,credits,description
0,10.0,70,120,PACE 111 (3),green,white,1,12px,1,1,general,PACE 111,3,
1,80.0,115,120,LIBS 150 (1),green,white,1,12px,1,2,general,LIBS 150,1,
2,150.0,160,120,WRTG 111 (3),green,white,1,12px,1,3,general,WRTG 111,3,
3,310.0,70,120,WRTG 112 (3),green,white,1,12px,2,1,general,WRTG 112,3,
4,380.0,115,120,NUTR 100 (3),green,white,1,12px,2,2,general,NUTR 100,3,
5,450.0,160,120,BMGT 110 (3),blue,white,1,12px,2,3,major,BMGT 110,3,
6,540.0,70,120,SPCH 100 (3),green,white,1,12px,3,1,general,SPCH 100,3,
7,610.0,115,120,STAT 200 (3),red,white,1,12px,3,2,required,STAT 200,3,
8,840.0,70,120,IFSM 300 (3),red,white,1,12px,4,1,required,IFSM 300,3,
9,840.0,115,120,ACCT 220 (3),blue,white,1,12px,4,1,major,ACCT 220,3,


In [118]:
'SUMMER 2024'.capitalize()

'Summer 2024'

In [120]:
terms_remaining = max(headers.period)
terms_remaining

14

In [125]:
credits_next_term = headers.loc[headers['period'] == 1, 'credits'].values[0]
credits_next_term

7

In [122]:
completion_date = headers.loc[headers['period'] == terms_remaining, 'name'].values[0].capitalize()
completion_date

'Summer 2027'

In [126]:
completion_date.upper()

'SUMMER 2027'

In [123]:
total_credits_remaining = df['credits'].sum()
total_credits_remaining

120

In [60]:
headers = generate_header_data('SPRING 2024', 14)

df['width'] = 120
# Calculate 'x' column
df = pd.merge(df, headers[['period','x']], on='period', how='left')
df['x'] += 70*(df['session']-1)

# Calculate 'y' column
df = df.sort_values(by=['period', 'session', 'seq' ])
df['y_row'] = df.groupby('period').cumcount() + 1
df['y'] = 70 + 45 * (df['y_row'] - 1)

# Create rectangle colors
df[['color', 'textcolor']] = df.apply(set_colors, axis=1)

# Set text offset multiplier to 1 and text fontsize
df['offset'] = 1
df['fontsize'] = '12px'

df = df[['x', 'y', 'width', 'name', 'credits', 'color', 'textcolor', 'offset', 
         'fontsize', 'period', 'session']]

In [62]:
df

,x,y,width,name,credits,color,textcolor,offset,fontsize,period,session
0,10.0,70,120,PACE 111,3,green,white,1,12px,1,1
1,80.0,115,120,LIBS 150,1,green,white,1,12px,1,2
2,150.0,160,120,WRTG 111,3,green,white,1,12px,1,3
3,310.0,70,120,WRTG 112,3,green,white,1,12px,2,1
4,380.0,115,120,NUTR 100,3,green,white,1,12px,2,2
5,450.0,160,120,BMGT 110,3,blue,white,1,12px,2,3
6,540.0,70,120,SPCH 100,3,green,white,1,12px,3,1
7,610.0,115,120,STAT 200,3,red,white,1,12px,3,2
8,840.0,70,120,IFSM 300,3,red,white,1,12px,4,1
9,840.0,115,120,ACCT 220,3,blue,white,1,12px,4,1
